In [117]:
from pubg_util import mysql, load_schema, notifier
from sphynx import sphynx, NODE_SMALL, NODE_MEDIUM, NODE_LARGE
from pyspark.sql.functions import *
import pandas as pd
import pickle

spark = sphynx.get_spark(executor_count=4, app_name='west0p', node_spec=NODE_LARGE)

Spark cluster not assigned. creating a new one...
Node spec: 4 executors with 48G RAM each
Job Port 4049 is assigned for requested cluster
Waiting for Spark master to be available...
Spark master launched!
Creating new Spark session, name: west0p...
Waiting for all executors ready...
All executors connected!
Complete! elapsed time: 00:00:25


In [116]:
sphynx.stop()

Stopping Spark session...


/usr/local/spark/python/pyspark/context.py:446: RuntimeWarning: Unable to cleanly shutdown Spark JVM process. It is possible that the process has crashed, been killed or may also be in a zombie state.
  RuntimeWarning


Destroying Spark cluster...
Done!
You can safely ignore the warning message if there is any.


In [29]:
cash_mtx = load_data_mart("pc", "2022-01-12", "2022-06-19", "cash_mtx")

In [31]:
pu_master = load_data_mart("pc", "2022-06-19", "2022-06-19", "pu_master").select("account_id", "first_mtx_date").withColumnRenamed("account_id", "account_id_")

In [32]:
starter = cash_mtx.where(col("product_id") == "gcoinbundledesc.1016")
starter_npu = starter.join(pu_master, (pu_master.first_mtx_date == starter.date) & (pu_master.account_id_ == starter.account_id), "left")

In [33]:
starter_npu = starter_npu.withColumn("is_npu", when(~col("first_mtx_date").isNull(), "true").otherwise("false"))

In [34]:
starter_npu_list = starter_npu.where(col("is_npu") == "true").select("account_id")
starter_npu_cash = cash_mtx.where(col("product_id") != "gcoinbundledesc.1016").join(starter_npu_list, "account_id")

In [64]:
starter_npu = starter_npu.withColumnRenamed("date", "starter_date").withColumnRenamed("time", "starter_buy_time")
starter_npu_cash = starter_npu_cash.groupBy("account_id").agg(min("date").alias("first_date"), min("time").alias("first_buy_time")).withColumnRenamed("account_id", "account_id_c")

In [67]:
starter_cash_df = starter_npu.join(starter_npu_cash, (starter_npu.account_id == starter_npu_cash.account_id_c) & (starter_npu.starter_buy_time < starter_npu_cash.first_buy_time), "left")

In [69]:
starter_cash_df = starter_cash_df.withColumn("date_diff", datediff(col("starter_buy_time"), col("first_buy_time")))

In [55]:
starter_cash_table = starter_cash_df.groupBy("starter_date", "is_npu").agg(countDistinct("account_id").alias("pu"), count("first_buy_time").alias("buy_after_starter"), mean("date_diff").alias("avg_date_diff"))

In [68]:
# starter_cash_df2 = starter_cash_df.withColumn("buy_other_cash", when(col("first_buy_time").isNull(), "false").otherwise("true"))

In [71]:
starter_cash_table2 = starter_cash_df.withColumn("in_28Day", when(-col("date_diff") < 28, 1).otherwise(0)). \
    groupBy("starter_date", "is_npu", "in_28Day").agg(countDistinct("account_id").alias("pu"), count("first_buy_time").alias("buy_after_starter"), sum("in_28Day").alias("in_28Day_pu"))

In [56]:
mysql.insert_table(starter_cash_table, "labs", "starter_cash_df")

In [72]:
mysql.insert_table(starter_cash_table2, "labs", "starter_cash_df2")

In [13]:
first_2month_npu_list = starter_npu.where((col("date") < "2022-03-12") & (col("is_npu") == "true")).select("account_id")
cash = load_data_mart("pc", "2022-05-04", "2022-06-28", "cash_mtx").join(first_2month_npu_list, "account_id")
gcoin = load_data_mart("pc", "2022-05-04", "2022-06-28", "gcoin_use").join(first_2month_npu_list, "account_id")

In [14]:
first_2month_npu_list.count()

715719

In [15]:
cash.select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                     58344|
+--------------------------+



In [24]:
58344/715719*100

8.151802592917052

In [20]:
cash.groupBy("product_name", "price_").agg(count("*"), countDistinct("account_id")).show(truncate=False)

+---------------------------+------+--------+--------------------------+
|product_name               |price_|count(1)|count(DISTINCT account_id)|
+---------------------------+------+--------+--------------------------+
|500(+10 Bonus) G-Coin      |5.00  |967     |789                       |
|5000(+500 Bonus) G-Coin    |49.99 |5024    |3225                      |
|10000(+1200 Bonus) G-Coin  |99.99 |3375    |1514                      |
|HARDCORE HITMAN VALUE PACK |9.55  |78      |78                        |
|10000(+1200 Bonus) G-Coin  |100.00|196     |99                        |
|2500(+200 Bonus) G-Coin    |24.99 |12318   |8112                      |
|100 G-Coin                 |1.00  |1025    |610                       |
|1000(+50 Bonus) G-Coin     |10.00 |1800    |1252                      |
|2500(+200 Bonus) G-Coin    |25.00 |775     |592                       |
|CYBER THREAT AWM VALUE PACK|9.99  |24734   |24734                     |
|HARDCORE HITMAN VALUE PACK |9.99  |1161    |1161  

In [16]:
gcoin.select(countDistinct("account_id")).show()

+--------------------------+
|count(DISTINCT account_id)|
+--------------------------+
|                    102879|
+--------------------------+



In [25]:
102879/715719*100

14.374216696776248

In [22]:
gcoin.groupBy("event_name").agg(count("*"), countDistinct("account_id"), sum(col("free_use")), sum(col("paid_use")), sum(col("free_use") + col("paid_use"))).show(truncate=False)

+-----------------------+--------+--------------------------+-------------+-------------+--------------------------+
|event_name             |count(1)|count(DISTINCT account_id)|sum(free_use)|sum(paid_use)|sum((free_use + paid_use))|
+-----------------------+--------+--------------------------+-------------+-------------+--------------------------+
|202105_explorer_emote  |201     |201                       |45920        |54580        |100500                    |
|202205_bloom           |5745    |5194                      |1255170      |995710       |2250880                   |
|202108_face            |2129    |2068                      |567380       |497120       |1064500                   |
|202201_valentine_emote |508     |501                       |126830       |127170       |254000                    |
|202107_emote           |354     |353                       |84880        |64920        |149800                    |
|202205_bduck           |14994   |10882                     |553

In [30]:
cash_mtx.where(col("date") >= "2022-05-01").groupBy("product_name").agg(countDistinct("account_id").alias("pu"), sum("price_").alias("revenue")).orderBy("pu", ascending=False).show(truncate=False)

+-------------------------------------+------+----------+
|product_name                         |pu    |revenue   |
+-------------------------------------+------+----------+
|STARTER PACKAGE                      |264132|524503.40 |
|CYBER THREAT AWM VALUE PACK          |259526|2587247.90|
|1000(+50 Bonus) G-Coin               |250341|3639295.76|
|500(+10 Bonus) G-Coin                |207201|1351761.04|
|100 G-Coin                           |190142|347891.81 |
|2500(+200 Bonus) G-Coin              |141524|5427157.22|
|BATTLEGROUNDS PLUS                   |70001 |908839.39 |
|5000(+500 Bonus) G-Coin              |66676 |5085729.43|
|10000(+1200 Bonus) G-Coin            |36917 |9841771.81|
|HARDCORE HITMAN VALUE PACK           |18915 |188420.09 |
|GILT ELEGANCE - 1050 G-COIN SKIN PACK|16    |159.84    |
+-------------------------------------+------+----------+



In [39]:
starter_npu_list = starter_npu.where((col("is_npu") == "true") & (col("date") >= "2022-05-23")).select("account_id")
starter_npu_cash = cash_mtx.where(col("product_id") != "gcoinbundledesc.1016").join(starter_npu_list, "account_id")

In [40]:
starter_npu_cash_pd = starter_npu_cash.where(col("date") >= "2022-05-23").groupBy("product_name").agg(countDistinct("account_id").alias("pu"), sum("price_").alias("revenue")).toPandas()

In [41]:
starter_npu_cash_pd.sort_values("pu")

,product_name,pu,revenue
5,10000(+1200 Bonus) G-Coin,470,110189.63
4,5000(+500 Bonus) G-Coin,1184,84285.04
6,2500(+200 Bonus) G-Coin,3213,115557.43
1,1000(+50 Bonus) G-Coin,6329,88757.34
7,CYBER THREAT AWM VALUE PACK,6402,63804.62
3,BATTLEGROUNDS PLUS,8080,104916.48
0,500(+10 Bonus) G-Coin,8734,54906.60
2,100 G-Coin,18772,32139.44


In [43]:
target_date = "2022-06-29"
user_master = load_data_mart("pc", target_date, target_date, table_name="user_master")

In [47]:
def load_membership(game, device, start_date, end_date) :
    sc = spark.sparkContext
    jvm = sc._jvm
    conf = sc._jsc.hadoopConfiguration()
    startdate = datetime.strptime(start_date, '%Y-%m-%d')
    enddate = datetime.strptime(end_date, '%Y-%m-%d')
    date_list = [(startdate + timedelta(d)).strftime("%Y-%m-%d") for d in range((enddate-startdate).days+1)]   
    path_list = ["s3a://pubg-log-labs/data_mart/{game}/membership/{device}/{target_date}" \
            .format(game = game, device = device, target_date = target_date) for target_date in date_list]
    true_path = []
    if device == '*' :
        all_device = ['pc', 'console', 'lpc']
        for devices in all_device :
            for path in path_list :
                true_path.append(path.replace("*",devices))
    else :
        true_path = path_list
    read_path = []
    for real_path in true_path : ###실제 있는 경로리스트 추출
        uri = jvm.java.net.URI(real_path)
        fs = jvm.org.apache.hadoop.fs.FileSystem.get(uri, conf)
        if fs.exists(jvm.org.apache.hadoop.fs.Path(real_path)) :
            read_path.append(str(real_path))       
    try : #####
        df = spark.read.parquet(*read_path)
    except : ##### 경로가 없으면 임의로 만들어줌
        df = spark.read.parquet("s3a://pubg-log-labs/data_mart/pubg_test/membership/pc/2022-01-04") \
            .where("date = '2099-01-01'")
    return df

load_membership("pubg", "pc", target_date, target_date).show(truncate=False)

+----+----------+------+----+------+--------+----------+---------------+----------------------+
|date|account_id|net_id|game|device|platform|membership|conversion_time|get_legacy_reward_time|
+----+----------+------+----+------+--------+----------+---------------+----------------------+
+----+----------+------+----+------+--------+----------+---------------+----------------------+



In [71]:
user = load_data_mart("pc", "2022-06-01", "2022-06-28", "user_master").where((col("firstlogindate") >= "2022-06-01") & (col("firstlogindate") <= "2022-06-14"))

In [73]:
cash = load_data_mart("pc", "2022-06-01", "2022-06-28", "cash_mtx").where(col("product_id") == "gcoinbundledesc.1016")

In [74]:
starter_daily = cash.groupBy("date").agg(count("*").alias("cnt")).toPandas()

In [75]:
starter_new_daily = cash.join(user.withColumnRenamed("accountid", "account_id").select("account_id"), "account_id").groupBy("date").agg(count("*").alias("new_user_cnt")).toPandas()

In [76]:
df = pd.merge(starter_daily, starter_new_daily,on = "date").sort_values("date")

In [77]:
df

,date,cnt,new_user_cnt
25,2022-06-01,4797,208
3,2022-06-02,5090,358
8,2022-06-03,5539,442
22,2022-06-04,5574,542
4,2022-06-05,5204,558
2,2022-06-06,4481,464
13,2022-06-07,4398,507
16,2022-06-08,7356,621
14,2022-06-09,6392,635
17,2022-06-10,5716,620


In [78]:
df['percentage'] = df['new_user_cnt']/df["cnt"]

In [79]:
df

,date,cnt,new_user_cnt,percentage
25,2022-06-01,4797,208,0.043360
3,2022-06-02,5090,358,0.070334
8,2022-06-03,5539,442,0.079798
22,2022-06-04,5574,542,0.097237
4,2022-06-05,5204,558,0.107225
2,2022-06-06,4481,464,0.103548
13,2022-06-07,4398,507,0.115280
16,2022-06-08,7356,621,0.084421
14,2022-06-09,6392,635,0.099343
17,2022-06-10,5716,620,0.108467


In [80]:
user2 = load_data_mart("pc", "2022-06-28", "2022-06-28", "user_master").where(col("lastlogindate") >= "2022-06-01").withColumnRenamed("accountid", "account_id").select("account_id", "firstlogindate")
sp_df = cash.join(user2, "account_id")

In [93]:
sp_df = sp_df.withColumn("month_diff", round(months_between("date", "firstlogindate")))

In [96]:
sp_pd = sp_df.groupBy("date", "month_diff").agg(countDistinct("account_id").alias("pu"))

In [97]:
sp_pd_2 = sp_df.groupBy("date").agg(countDistinct("account_id").alias("total_pu"))

In [98]:
mysql.insert_table(sp_pd, "labs", "starter_new_user_pu")

In [ ]:
sp_final = pd.merge(sp_pd, sp_pd_2, on="date").sort_values(["date", "month_diff"])

In [ ]:
sp_final.month_diff = sp_final.month_diff.round()

In [92]:
sp_final[sp_final.month_diff <= 1]

,date,month_diff,pu,total_pu
27399,2022-06-01,0.0,208,4797
27341,2022-06-01,0.0,110,4797
27441,2022-06-01,0.0,57,4797
26932,2022-06-01,0.0,59,4797
27353,2022-06-01,0.0,35,4797
26797,2022-06-01,0.0,30,4797
27197,2022-06-01,0.0,38,4797
27383,2022-06-01,0.0,35,4797
26860,2022-06-01,0.0,23,4797
27784,2022-06-01,0.0,26,4797


최근 1개월간 NPU들의 첫 구매 현금 상품

In [101]:
cash_mtx = load_data_mart("pc", "2022-06-01", "2022-06-28", "cash_mtx")
pu_master = load_data_mart("pc", "2022-06-28", "2022-06-28", "pu_master").select("account_id", "first_mtx_date")
cash_mtx = cash_mtx.join(pu_master, "account_id")

In [103]:
npu_cash_mtx = cash_mtx.where(col("date") == col("first_mtx_date"))

In [104]:
npu_cash_df = npu_cash_mtx.groupBy("date", "product_name").agg(countDistinct("account_id").alias("pu"), sum("price_").alias("revenue"))

In [105]:
mysql.drop_tablesert_table(npu_cash_df, "labs", "npu_cash_df")

In [109]:
npu_cash_daily_min_time = npu_cash_mtx.groupBy("date", "account_id").agg(min("time").alias("time"))

In [114]:
npu_cash_df2 = npu_cash_mtx.join(npu_cash_daily_min_time, ["date", "account_id", "time"]).groupBy("date", "product_name").agg(countDistinct("account_id").alias("pu"), sum("price_").alias("revenue"))

In [115]:
mysql.insert_table(npu_cash_df2, "labs", "npu_cash_df2")

기존 PU들 중 Starter Package를 구매하지 않은 유저들

In [120]:
cash = load_data_mart("pc", "2022-01-12", "2022-06-30", "cash_mtx")
starter_pack = cash.where(col("product_id") == "gcoinbundledesc.1016")

In [131]:
pu_after_f2p = load_data_mart("pc", "2022-06-30", "2022-06-30", "pu_master").where(col("last_mtx_date") >= "2022-01-12").select("account_id", "first_mtx_date", "last_mtx_date")

In [132]:
no_buy_starter_pu = pu_after_f2p.join(starter_pack, "account_id", "left_anti")

In [133]:
no_buy_starter_pu.select(countDistinct("account_id").alias("user_cnt")).show(truncate=False)

+--------+
|user_cnt|
+--------+
|565196  |
+--------+



In [140]:
cash.join(no_buy_starter_pu, "account_id").groupBy("product_name").agg(countDistinct("account_id").alias("pu"), sum("price_").alias("revenue"), count("*").alias("cnt")).orderBy("pu").toPandas()

,product_name,pu,revenue,cnt
0,GILT ELEGANCE - 1050 G-COIN SKIN PACK,132,1318.68,132
1,CYBER THREAT AWM VALUE PACK,9752,97032.64,9752
2,HARDCORE HITMAN VALUE PACK,15550,154744.78,15550
3,10000(+1200 Bonus) G-Coin,20352,5017447.90,50179
4,5000(+500 Bonus) G-Coin,38943,2918560.40,58381
5,2500(+200 Bonus) G-Coin,98453,3726877.02,149129
6,100 G-Coin,115338,199649.93,201428
7,BATTLEGROUNDS PLUS,148719,1930742.22,148722
8,500(+10 Bonus) G-Coin,166637,1085166.39,217429
9,1000(+50 Bonus) G-Coin,188758,2684429.09,268676


In [141]:
cash.join(no_buy_starter_pu, "account_id").where(col("first_mtx_date") == col("date")).groupBy("product_name").agg(countDistinct("account_id").alias("pu"), sum("price_").alias("revenue"), count("*").alias("cnt")).orderBy('pu').toPandas()

,product_name,pu,revenue,cnt
0,GILT ELEGANCE - 1050 G-COIN SKIN PACK,83,829.17,83
1,CYBER THREAT AWM VALUE PACK,1733,17257.67,1733
2,HARDCORE HITMAN VALUE PACK,4077,40616.59,4077
3,10000(+1200 Bonus) G-Coin,7082,1149488.20,11496
4,5000(+500 Bonus) G-Coin,8034,468019.04,9362
5,2500(+200 Bonus) G-Coin,31678,855038.94,34214
6,100 G-Coin,52507,82794.89,83567
7,500(+10 Bonus) G-Coin,69593,380936.30,76329
8,1000(+50 Bonus) G-Coin,72053,793487.39,79419
9,BATTLEGROUNDS PLUS,138137,1793534.65,138139


In [135]:
membership = load_membership("pubg", "pc", "2022-06-30", "2022-06-30")

In [136]:
no_buy_starter_pu = no_buy_starter_pu.join(membership.select("membership", "account_id"), "account_id")

In [137]:
cash.join(no_buy_starter_pu, "account_id").groupBy("membership", "product_name").agg(countDistinct("account_id").alias("pu"), sum("price_").alias("revenue")).orderBy("membership", "pu").toPandas()

,membership,product_name,pu,revenue
0,basic,GILT ELEGANCE - 1050 G-COIN SKIN PACK,8,79.92
1,basic,BATTLEGROUNDS PLUS,12,155.24
2,basic,CYBER THREAT AWM VALUE PACK,956,9481.36
3,basic,10000(+1200 Bonus) G-Coin,1041,144989.27
4,basic,HARDCORE HITMAN VALUE PACK,1325,13145.23
5,basic,5000(+500 Bonus) G-Coin,3623,221169.99
6,basic,2500(+200 Bonus) G-Coin,15364,469172.08
7,basic,100 G-Coin,24470,43394.17
8,basic,500(+10 Bonus) G-Coin,33383,202213.97
9,basic,1000(+50 Bonus) G-Coin,33935,428945.80
